In [1]:
!pip install transformers datasets trl peft bitsandbytes accelerate torch sentencepiece huggingface_hub -U
!pip install --upgrade trl transformers accelerate
!pip install --upgrade deepspeed==0.14.4

# ATTENTION: Be sure to restart the notebook after installing all packages!

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 144.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 37.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 9.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 136.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 66.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 110

In [1]:
from huggingface_hub import interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import warnings
from IPython.display import display, Markdown

# Suppress warnings if needed
warnings.filterwarnings("ignore")

# --- Configuration ---
base_model_id = "meta-llama/Llama-3.2-3B-Instruct"
# *** IMPORTANT: Update this path to where your LoRA adapter weights are saved ***
adapter_path = "./results_llama3_finetuned"
# Set device (use "cuda" if GPU is available, otherwise "cpu")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Optional: Quantization config for loading base model (if needed for memory)
# Adjust based on your inference hardware and how you trained/saved.
# If you merge first, you might load the base without quantization here.
use_quantization = False # Set to True if you want to load base model in 4-bit
compute_dtype = torch.bfloat16 # Or torch.float16

bnb_config = None
if use_quantization:
    print("Loading base model with 4-bit quantization...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False, # Typically False for inference
    )
else:
    print(f"Loading base model with dtype: {compute_dtype}")


# --- Load Base Model and Tokenizer ---
print(f"Loading base model: {base_model_id}")
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    torch_dtype=compute_dtype if not use_quantization else None, # Set dtype if not quantizing
    device_map="auto", # Automatically handle device placement
    trust_remote_code=True,
)

print(f"Loading tokenizer for: {base_model_id}")
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # Set pad token if needed
tokenizer.padding_side = "right"

# --- Load LoRA Adapter and Merge ---
print(f"Loading LoRA adapter from: {adapter_path}")
# Load the LoRA adapter weights on top of the base model
# Note: device_map="auto" should handle placing the adapter correctly
model = PeftModel.from_pretrained(model, adapter_path)
print("LoRA adapter loaded.")

print("Merging LoRA adapter into the base model...")
# Merge the adapter weights into the base model weights
# This returns a standard transformers model object
model = model.merge_and_unload()
print("Adapter merged.")

# Set the model to evaluation mode
model.eval()

def get_response(prompt_text):
    # --- Prepare Prompt using the Correct Chat Template ---
    # *** CRITICAL: Use the exact same format as your training data ***
    # Example Llama 3.2 Instruct format (VERIFY THIS STRUCTURE!)
    # prompt_text = "Investigating the effects of manipulation on the gale shapley algorithm."

    # Format the prompt using the chat template structure
    # Ensure this matches the format_dataset_batch function from your training script
    formatted_prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

    {prompt_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

    """ # Note the trailing space and newline for the assistant's turn

    print("\nFormatted Prompt:")
    print(formatted_prompt)

    # --- Tokenize Input ---
    print("Tokenizing prompt...")
    # Encode the formatted prompt into token IDs
    # Ensure tensors are moved to the same device as the model
    inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=False, truncation=True).to(device)

    # --- Generate Output ---
    print("Generating response...")
    # Generate text using the model
    # Adjust generation parameters as needed (max_new_tokens, temperature, top_p, etc.)
    with torch.no_grad(): # Disable gradient calculations for inference
        outputs = model.generate(
            **inputs,
            max_new_tokens=2048,          # Maximum number of new tokens to generate
            do_sample=True,             # Whether to use sampling; set to False for deterministic output
            temperature=0.6,            # Controls randomness (lower = more deterministic)
            top_p=0.9,                  # Nucleus sampling probability
            eos_token_id=tokenizer.eos_token_id # Stop generation when EOS token is encountered
        )

    # --- Decode and Print Output ---
    # Decode the generated token IDs back into text
    # Skip special tokens (like padding or EOS) in the output string
    # We slice the output to only decode the newly generated tokens (after the input prompt)
    generated_token_ids = outputs[0][inputs['input_ids'].shape[1]:]
    response = tokenizer.decode(generated_token_ids, skip_special_tokens=True)

    return response

# response = get_response("Investigating the effects of manipulation on the gale shapley algorithm.")
# div = "="*80
# print(f"\n{div}\nModel Response:")
# display(Markdown(response))


# print(response)

# print("\nInference complete.")



/usr/local/lib/python3.11/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.11/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-04-23 16:45:30.018161: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-23 16:45:30.018224: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-23 16:45:30.019583: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:

[2025-04-23 16:45:31,565] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.7
 [WARNING]  using untested triton version (3.3.0), only 1.0.0 is known to be compatible


/usr/local/lib/python3.11/dist-packages/deepspeed/runtime/zero/linear.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @autocast_custom_fwd
/usr/local/lib/python3.11/dist-packages/deepspeed/runtime/zero/linear.py:66: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @autocast_custom_bwd


Using device: cuda
Loading base model with dtype: torch.bfloat16
Loading base model: meta-llama/Llama-3.2-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading tokenizer for: meta-llama/Llama-3.2-3B-Instruct
Loading LoRA adapter from: ./results_llama3_finetuned
LoRA adapter loaded.
Merging LoRA adapter into the base model...
Adapter merged.


In [ ]:
import pandas as pd
from tqdm import tqdm

test_data = pd.read_csv('test_questions.csv')
prompts = test_data['prompt'].tolist()
responses = []

for prompt in tqdm(prompts):
    response = get_response(prompt)
    responses.append(response)

# div = "="*80
# print(f"\n{div}\nModel Response:")
# display(Markdown(response))

test_data.insert(1, 'finetune_response', responses)

In [ ]:
test_data = test_data[['topic', 'prompt', 'completion', 'finetune_response']]

test_data.to_csv('eval_data.csv', index=False)


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import warnings
from IPython.display import display, Markdown

# Suppress warnings if needed
warnings.filterwarnings("ignore")

# --- Configuration ---
base_model_id = "meta-llama/Llama-3.2-3B-Instruct"
# *** IMPORTANT: Update this path to where your LoRA adapter weights are saved ***
adapter_path = "./results_llama3_finetuned"
# Set device (use "cuda" if GPU is available, otherwise "cpu")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Optional: Quantization config for loading base model (if needed for memory)
# Adjust based on your inference hardware and how you trained/saved.
# If you merge first, you might load the base without quantization here.
use_quantization = False # Set to True if you want to load base model in 4-bit
compute_dtype = torch.bfloat16 # Or torch.float16

bnb_config = None
if use_quantization:
    print("Loading base model with 4-bit quantization...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False, # Typically False for inference
    )
else:
    print(f"Loading base model with dtype: {compute_dtype}")


# --- Load Base Model and Tokenizer ---
print(f"Loading base model: {base_model_id}")
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    torch_dtype=compute_dtype if not use_quantization else None, # Set dtype if not quantizing
    device_map="auto", # Automatically handle device placement
    trust_remote_code=True,
)

print(f"Loading tokenizer for: {base_model_id}")
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # Set pad token if needed
tokenizer.padding_side = "right"

# --- Load LoRA Adapter and Merge ---
# print(f"Loading LoRA adapter from: {adapter_path}")
# Load the LoRA adapter weights on top of the base model
# Note: device_map="auto" should handle placing the adapter correctly
# model = PeftModel.from_pretrained(model, adapter_path)
# print("LoRA adapter loaded.")

# print("Merging LoRA adapter into the base model...")
# Merge the adapter weights into the base model weights
# This returns a standard transformers model object
# model = model.merge_and_unload()
# print("Adapter merged.")

# Set the model to evaluation mode
model.eval()

def get_response(prompt_text):
    # --- Prepare Prompt using the Correct Chat Template ---
    # *** CRITICAL: Use the exact same format as your training data ***
    # Example Llama 3.2 Instruct format (VERIFY THIS STRUCTURE!)
    # prompt_text = "Investigating the effects of manipulation on the gale shapley algorithm."

    # Format the prompt using the chat template structure
    # Ensure this matches the format_dataset_batch function from your training script
    formatted_prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

    {prompt_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

    """ # Note the trailing space and newline for the assistant's turn

    # print("\nFormatted Prompt:")
    # print(formatted_prompt)

    # --- Tokenize Input ---
    # print("Tokenizing prompt...")
    # Encode the formatted prompt into token IDs
    # Ensure tensors are moved to the same device as the model
    inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=False, truncation=True).to(device)

    # --- Generate Output ---
    # print("Generating response...")
    # Generate text using the model
    # Adjust generation parameters as needed (max_new_tokens, temperature, top_p, etc.)
    with torch.no_grad(): # Disable gradient calculations for inference
        outputs = model.generate(
            **inputs,
            max_new_tokens=2048,          # Maximum number of new tokens to generate
            do_sample=True,             # Whether to use sampling; set to False for deterministic output
            temperature=0.6,            # Controls randomness (lower = more deterministic)
            top_p=0.9,                  # Nucleus sampling probability
            eos_token_id=tokenizer.eos_token_id # Stop generation when EOS token is encountered
        )

    # --- Decode and Print Output ---
    # Decode the generated token IDs back into text
    # Skip special tokens (like padding or EOS) in the output string
    # We slice the output to only decode the newly generated tokens (after the input prompt)
    generated_token_ids = outputs[0][inputs['input_ids'].shape[1]:]
    response = tokenizer.decode(generated_token_ids, skip_special_tokens=True)

    return response

# response = get_response("Investigating the effects of manipulation on the gale shapley algorithm.")
# div = "="*80
# print(f"\n{div}\nModel Response:")
# display(Markdown(response))


# print(response)

# print("\nInference complete.")



/usr/local/lib/python3.11/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.11/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-04-23 19:16:58.997342: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-23 19:16:58.997395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-23 19:16:58.998774: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:

[2025-04-23 19:17:00,557] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.7
 [WARNING]  using untested triton version (3.3.0), only 1.0.0 is known to be compatible


/usr/local/lib/python3.11/dist-packages/deepspeed/runtime/zero/linear.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @autocast_custom_fwd
/usr/local/lib/python3.11/dist-packages/deepspeed/runtime/zero/linear.py:66: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @autocast_custom_bwd


Using device: cuda
Loading base model with dtype: torch.bfloat16
Loading base model: meta-llama/Llama-3.2-3B-Instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading tokenizer for: meta-llama/Llama-3.2-3B-Instruct


In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

test_data = pd.read_csv('eval_data.csv')
prompts = test_data['prompt'].tolist()
base_responses = []

for prompt in tqdm(prompts):
    response = get_response(prompt)
    base_responses.append(response)

# div = "="*80
# print(f"\n{div}\nModel Response:")
# display(Markdown(response))

test_data.insert(-1, 'base_response', base_responses)

  0%|          | 0/122 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

ValueError: unbounded slice